In [122]:
import cv2
import numpy as np
import matplotlib.pyplot as plt 
from skimage.util import random_noise
import rawpy
import imageio

%matplotlib inline

In [123]:
class SimulateCondition:
    def __init__(self, image):
        self.image = image
        
#         self.GAMMA = np.random.uniform(0.4, 0.7, 1)[0]
        self.GAMMA = np.random.normal(0.9, 0.1)
        self.NOISE_PEAK = 0.5
#         self.WB = np.random.uniform(0.7, 1.0, 3)
#         self.WB = [np.random.normal(0.8, 0.04), np.random.normal(0.82, 0.04), np.random.normal(1.42, 0.04)]
        self.WB = [np.random.normal(0.8, 0.04), np.random.normal(0.8, 0.04), np.random.normal(0.7, 0.04)]
#         self.WB = [np.random.normal(0.4, 0.04), np.random.normal(0.9, 0.04), np.random.normal(0.7, 0.04)]
        self.BL = 30
        
        self.img_rows, self.img_cols, self.img_colors = image.shape
        
    def adjust_gamma(self, image):
        invGamma = 1.0 / self.GAMMA
        table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
        print("> Applied gamma correction")
        return cv2.LUT(image, table)
    
    def add_noise(self, image):
        #noisy = np.random.poisson(image / 255.0 * self.NOISE_PEAK) / self.NOISE_PEAK * 255
#         noisy = np.random.poisson(image)
#         noisy = np.uint8(image + noisy*self.NOISE_PEAK)
        noisy = np.uint8(random_noise(image, mode='poisson') * 255.0)
        print("> Noise Added")
        return noisy
    
    def remove_white_balance(self, image):
        white_unbalanced =  np.zeros((self.img_rows,self.img_cols,self.img_colors), dtype='uint8')
        white_unbalanced[:,:,0] = np.int8(self.WB[0] * image[:,:,0])
        white_unbalanced[:,:,1] = np.int8(self.WB[1] * image[:,:,1])
        white_unbalanced[:,:,2] = np.int8(self.WB[2] * image[:,:,2])
        print("> White unbalanced")
        return white_unbalanced
    
    def increase_black_level(self, image):
        bl_image = np.copy(image)
        bl_image[bl_image<self.BL] = self.BL
        bl_image = bl_image - self.BL
        print("> Black level increased")
        return bl_image
    
    def low_light(self, gamma_correction=True, white_balance=True, noise=True, black_level=True):
        if gamma_correction:
            image = self.adjust_gamma(self.image)             #image with gamma correction
        if white_balance:
            image = self.remove_white_balance(image)     #image with gamma correction, noise, reduced white balance  
        if noise:
            image = self.add_noise(image)                     #image with gamma correction, noise
        if black_level:
            image = self.increase_black_level(image)         #image with gamma correction, noise, reduced white balance, added black level
            
        return image     

In [124]:
def show_image(image, convert=False):
    if convert:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.show()

In [125]:
# img = cv2.imread('./flickr30K_images/flickr30K_images/13648274.jpg')
# img = cv2.imread('./flickr30K_images/flickr30K_images/2069887.jpg')
# img = cv2.imread('./a0014-WP_CRW_6320.dng')
img = cv2.imread('trashcan.jpg')
# img = rawpy.imread('./sony_image.ARW').postprocess(use_camera_wb=True)
# img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
img.shape

(1024, 1024, 3)

## GAMMA

In [126]:
sim = SimulateCondition(img)

In [127]:
ll_img = sim.low_light(noise=False, white_balance=False, black_level=False)

> Applied gamma correction


In [128]:
cv2.imwrite('./results/ex_12.png', img)

True

In [129]:
cv2.imwrite('./results/ex_12_gamma.png', ll_img)

True

## GAMMA + WHITE UNBALANCED

In [130]:
sim = SimulateCondition(img)
ll_img = sim.low_light(noise=False, black_level=False)
cv2.imwrite('./results/ex_12_gamma_wb.png', ll_img)

> Applied gamma correction
> White unbalanced


True

## GAMMA + WHITE UNBALANCED + NOISE

In [131]:
sim = SimulateCondition(img)
ll_img = sim.low_light(black_level=False)
cv2.imwrite('./results/ex_12_gamma_wb_noise.png', ll_img)

> Applied gamma correction
> White unbalanced
> Noise Added


True

## GAMMA + WHITE UNBALANCED + NOISE + BLACK LEVEL

In [132]:
sim = SimulateCondition(img)
ll_img = sim.low_light()
cv2.imwrite('./results/ex_12_gamma_wb_noise_bl.png', ll_img)

> Applied gamma correction
> White unbalanced
> Noise Added
> Black level increased


True